# Import

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime
import math

# myLibrary

In [2]:
#製造answer.csv的第1列
def answer_header():
    header = ['user_id']
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

In [3]:
def get_week(date_str):
    d = get_date(date_str)
    w = datetime.datetime.strftime(d, '%w')
    return int(w)

In [4]:
def get_date(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

In [5]:
def get_slot(time_str):
    the_time = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
    time_sec0 = datetime.datetime.strptime('01:00:00.00', '%H:%M:%S.%f')
    time_sec1 = datetime.datetime.strptime('09:00:00.00', '%H:%M:%S.%f')
    time_sec2 = datetime.datetime.strptime('17:00:00.00', '%H:%M:%S.%f')
    time_sec3 = datetime.datetime.strptime('21:00:00.00', '%H:%M:%S.%f')
    time_sec4 = datetime.datetime.strptime('23:59:59.99', '%H:%M:%S.%f')
    if time_sec0 <= the_time and the_time <= time_sec1:
        slot = 0
    elif time_sec1 <= the_time and the_time <= time_sec2:
        slot = 1
    elif time_sec2 <= the_time and the_time <= time_sec3:
        slot = 2
    elif time_sec3 <= the_time and the_time <= time_sec4:
        slot = 3
    else:
        slot = 4
    return slot

In [6]:
def get_days(end, start):
    d = end - start
    return d.days

In [7]:
def update_week(week, date_str, event_week, addition):
    date = get_date(date_str)
    end_date = get_date('2017-08-14')
    start_date = get_date('2017-05-13')
    if date > start_date:
        days = get_days(end_date, date)
        weight = 0
        for i in range(1,33):
            if (days/7) <= i:
                weight = math.pow(0.5, i)
                break
        addition = list(np.array(addition) * weight)
        week.days[event_week].add_slots(addition)
        another_addition = [0, 0, 0]
        another_addition.append(addition[4])
        week.days[event_week-1].add_slots(another_addition)
    return week

In [8]:
def error_list():
    errorlist= ['error', 'api error', 'limit playzone countdown']
    return errorlist

# myClass

In [9]:
class Day():
    def __init__(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def reset(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def get_slots(self):
        return [self.slot0, self.slot1, self.slot2, self.slot3]
    def add_slots(self, addition):
        self.slot0 += addition[0]
        self.slot1 += addition[1]
        self.slot2 += addition[2]
        self.slot3 += addition[3]

In [10]:
class Week():
    def __init__(self):
        self.days = []
        for i in range(7):
            self.days.append(Day())
    def reset(self):
        for i in range(7):
            self.days[i].reset()
    def get_answer(self, id):
        tmp = []
        for i in range(1, 7):
            tmp += self.days[i].get_slots()
        tmp += self.days[0].get_slots()
        answer = [id] + list(np.array(tmp))
        return answer

# Main

In [11]:
with open('answer_v28.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(answer_header())
    week = Week()
    # 開始處理資料
    for data_count in range(46, 76): # revise
        # 顯示進度
        print(data_count)
        data = pd.read_csv('private/data-0' + str(data_count) + '.csv')
        
        # 取得目前csv檔案的首尾user_id
        head = data.head(1)['user_id'].values[0]
        tail = data.tail(1)['user_id'].values[0]
        
        # 依序處理各id資料並寫入answer.csv
        for id in range(head, tail+1):
            id_data = data[data['user_id'] == id]
            length = len(id_data)
            
            # 依序處理某id的每一筆資料
            now_date = id_data['event_time'].values[0].split()[0]
            addition = [0, 0, 0, 0, 0]
            for i in range(length):
                # 切分時間獲得星期和slot
                event_time = id_data['event_time'].values[i].split()
                date = event_time[0]
                
                # 當日期變化
                if date != now_date:
                    # 將前日的紀錄更新進week
                    event_week = get_week(now_date)
                    week = update_week(week, now_date, event_week, addition)
                    #並初始化now_date和addition
                    now_date = date
                    addition = [0, 0, 0, 0, 0]
                
                # 若無error則把紀錄活動至addition
                trigger = id_data['action_trigger'].values[i]
                if trigger not in error_list():
                    event_slot = get_slot(event_time[1])
                    addition[event_slot] = 1
            
            # 補齊最後一段日期的紀錄
            event_week = get_week(now_date)
            week = update_week(week, now_date, event_week, addition)
            
            # 寫入answer.csv
            writer.writerow(week.get_answer(id))
            
            # 初始化week
            week.reset()
            
            #break # revise

# test

46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


# Test Area

In [12]:
id_data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
468053,94250,188854,9245992,69,2017-05-06 06:13:57.479,2788,seek,Web,1,12,online but unknown,False
468054,94250,188854,9245992,69,2017-05-06 06:14:00.071,102,seek,Web,1,12,online but unknown,False
468055,94250,188854,9245992,69,2017-05-06 06:15:50.557,161,seek,Web,1,12,online but unknown,False
468056,94250,188854,9245992,69,2017-05-06 06:15:52.292,46,seek,Web,1,12,online but unknown,False
468057,94250,188854,9245992,69,2017-05-06 06:15:57.796,4,seek,Web,1,12,online but unknown,False
468058,94250,188854,9245992,69,2017-05-06 06:22:43.417,403,video ended,Web,1,12,online but unknown,False
468059,94250,188854,9245992,69,2017-05-06 06:24:17.074,88,pause,Web,2,12,online but unknown,False
468060,94250,188854,9245993,69,2017-05-06 07:23:28.346,3475,next episode,Web,2,12,online but unknown,False
468061,94250,188854,9245993,69,2017-05-06 07:36:00.336,748,pause,Web,3,12,online but unknown,False
468062,94250,188854,9245993,69,2017-05-06 07:56:22.682,1195,pause,Web,3,12,online but unknown,False


In [13]:
data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,93913,188172,9206492,16,2017-01-05 14:01:01.248,2,interrupt,iOS,16,17,wifi,False
1,93913,188172,9206492,16,2017-01-05 14:02:06.008,55,leave,iOS,17,17,wifi,False
2,93913,188172,9206492,16,2017-01-05 14:19:23.126,1029,interrupt,iOS,16,17,wifi,False
3,93913,188172,9206492,16,2017-01-05 15:02:57.054,2587,leave,iOS,16,17,wifi,False
4,93913,188172,9206492,16,2017-01-05 15:19:36.894,992,interrupt,iOS,17,17,wifi,False
5,93913,188172,9206492,16,2017-01-05 15:21:17.121,7,seek,iOS,17,17,wifi,False
6,93913,188172,9206492,16,2017-01-05 15:21:21.060,2,seek,iOS,17,17,wifi,False
7,93913,188172,9206492,16,2017-01-05 15:21:25.139,2,seek,iOS,17,17,wifi,False
8,93913,188172,9206492,16,2017-01-05 15:51:49.170,1817,seek,iOS,17,17,wifi,False
9,93913,188172,9206492,16,2017-01-05 15:51:53.570,3,seek,iOS,17,17,wifi,False


# Check Answer

In [14]:
answer = pd.read_csv('answer_v28.csv')
answer

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,0.843506,0.843140,0.085327,0.151123,0.874878,0.991821,0.234009,0.262939,0.724487,...,0.597656,0.164062,0.196289,0.065674,0.003174,0.317627,0.757080,0.921143,0.668945,0.450195
1,57160,0.015625,0.765625,0.015625,0.000000,0.046875,0.796875,0.015625,0.000000,0.671875,...,0.023438,0.000000,0.000000,0.156250,0.015625,0.000000,0.000000,0.148438,0.257812,0.000000
2,57161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.125000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
3,57162,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.522461,0.000000,0.000977,0.008545,0.026123,0.000000,0.000244,0.125244,0.125000,0.000000
4,57163,0.250000,0.250000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,57164,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125000,0.000000,0.000000
6,57165,0.500000,0.581055,0.125977,0.001953,0.000000,0.961914,0.382812,0.330078,0.000000,...,0.000977,0.000000,0.313477,0.070312,0.250000,0.003906,0.254395,0.383301,0.250000,0.258301
7,57166,0.000000,0.750000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,...,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.125000,0.375000,0.250000,0.000000
8,57167,0.516479,0.894409,0.014648,0.000000,0.881714,0.929565,0.005859,0.000000,0.799683,...,0.503052,0.000000,0.268555,0.766846,0.018066,0.000488,0.777527,0.887756,0.004944,0.000000
9,57168,0.863159,0.991577,0.000000,0.961060,0.991699,0.929077,0.000000,1.124878,0.874878,...,0.000000,0.004883,0.208496,0.014648,0.000000,0.507874,0.090881,0.000061,0.000000,1.300171
